<a href="https://colab.research.google.com/github/theresaltrinh/project_voting/blob/main/voting_VA_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [180]:
! git clone https://github.com/theresaltrinh/project_voting

fatal: destination path 'project_voting' already exists and is not an empty directory.


In [181]:
import pandas as pd
import numpy as np

In [182]:
df = pd.read_csv("./project_voting/data/voting_VA.csv")
df.head()

,Unnamed: 0,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,11161,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,AL GORE,DEMOCRAT,5092,11925,20220315,TOTAL
1,11162,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,6352,11925,20220315,TOTAL
2,11163,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,RALPH NADER,GREEN,220,11925,20220315,TOTAL
3,11164,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,OTHER,OTHER,261,11925,20220315,TOTAL
4,11165,2000,VIRGINIA,VA,ALBEMARLE,51003,US PRESIDENT,AL GORE,DEMOCRAT,16255,36846,20220315,TOTAL


In [183]:
# Dropping 'state', 'stateAbbreviation, 'office', and 'version' columns due to 0 unique values

if df['state'].nunique() == 1:
    df.drop(columns=['state'], inplace=True)

if df['state_po'].nunique() == 1:
    df.drop(columns=['state_po'], inplace=True)

if df['office'].nunique() == 1:
    df.drop(columns=['office'], inplace=True)

if df['version'].nunique() == 1:
    df.drop(columns=['version'], inplace=True)

In [184]:
# Renaming columns
df = df.rename(columns = {'county_name': 'countyName',
                          'county_fips': 'countyCode',
                          'mode': 'voteType'}
                          )

# Correcting capitalization of column names
df['countyName'] = df['countyName'].str.capitalize()
df['candidate'] = df['candidate'].str.title()
df['party'] = df['party'].str.capitalize()
df['voteType'] = df['voteType'].str.capitalize()

# Dropping 'Unnamed: 0' column
df.drop('Unnamed: 0', axis=1, inplace=True)

In [185]:
df.head()

,year,countyName,countyCode,candidate,party,candidatevotes,totalvotes,voteType
0,2000,Accomack,51001,Al Gore,Democrat,5092,11925,Total
1,2000,Accomack,51001,George W. Bush,Republican,6352,11925,Total
2,2000,Accomack,51001,Ralph Nader,Green,220,11925,Total
3,2000,Accomack,51001,Other,Other,261,11925,Total
4,2000,Albemarle,51003,Al Gore,Democrat,16255,36846,Total


In [186]:
# Filtering by 2020 election and summing up candidate votes from all voting types to become "Total"

df_filtered = df[df['year'] == 2020]

df_2020 = df_filtered.groupby(['year', 'countyName', 'countyCode', 'candidate', 'party']).agg({'candidatevotes': 'sum', 'totalvotes': 'first'})

df_2020['voteType'] = 'Total'

df_2020.reset_index(inplace=True)

In [187]:
df_2020.head()

,year,countyName,countyCode,candidate,party,candidatevotes,totalvotes,voteType
0,2020,Accomack,51001,Donald J Trump,Republican,9172,16962,Total
1,2020,Accomack,51001,Jo Jorgensen,Libertarian,188,16962,Total
2,2020,Accomack,51001,Joseph R Biden Jr,Democrat,7578,16962,Total
3,2020,Accomack,51001,Other,Other,24,16962,Total
4,2020,Albemarle,51003,Donald J Trump,Republican,20804,64657,Total


In [188]:
# Dropping old 2020 rows from original dataframe and adding updated 2020 rows to dataframe

df = df[df['year'] != 2020]
df = pd.concat([df, df_2020], ignore_index=True)

In [189]:
df.head()

,year,countyName,countyCode,candidate,party,candidatevotes,totalvotes,voteType
0,2000,Accomack,51001,Al Gore,Democrat,5092,11925,Total
1,2000,Accomack,51001,George W. Bush,Republican,6352,11925,Total
2,2000,Accomack,51001,Ralph Nader,Green,220,11925,Total
3,2000,Accomack,51001,Other,Other,261,11925,Total
4,2000,Albemarle,51003,Al Gore,Democrat,16255,36846,Total


In [190]:
# Dropping 'voteType' column now that dataset reflects votes for total election

df = df.drop(columns=['voteType'])

In [192]:
df.head()

,year,countyName,countyCode,candidate,party,candidatevotes,totalvotes
0,2000,Accomack,51001,Al Gore,Democrat,5092,11925
1,2000,Accomack,51001,George W. Bush,Republican,6352,11925
2,2000,Accomack,51001,Ralph Nader,Green,220,11925
3,2000,Accomack,51001,Other,Other,261,11925
4,2000,Albemarle,51003,Al Gore,Democrat,16255,36846


In [193]:
df.columns

Index(['year', 'countyName', 'countyCode', 'candidate', 'party',
       'candidatevotes', 'totalvotes'],
      dtype='object')

In [195]:
# Setting temporary variable names

year = df['year']
countyName = df['countyName']
countyCode = df['countyCode']
candidate = df['candidate']
party = df['party']
candidatevotes = df['candidatevotes']
totalvotes = df['totalvotes']

In [196]:
# Looking at value counts for each variable

for var in df.columns:
  print(df[var].value_counts())

year
2000    536
2020    532
2004    402
2008    402
2012    402
2016    402
Name: count, dtype: int64
countyName
Fairfax              36
Richmond             36
Roanoke              36
Franklin             36
Bedford              36
                     ..
Galax city            4
Hampton city          4
Harrisonburg city     4
Hopewell city         4
Winchester city       4
Name: count, Length: 167, dtype: int64
countyCode
51001    20
51177    20
51540    20
51530    20
51520    20
         ..
51079    20
51077    20
51075    20
51840    20
51515    16
Name: count, Length: 134, dtype: int64
candidate
Other                803
George W. Bush       268
Barack Obama         268
Al Gore              134
Ralph Nader          134
John Kerry           134
John Mccain          134
Mitt Romney          134
Hillary Clinton      134
Donald Trump         134
Donald J Trump       133
Jo Jorgensen         133
Joseph R Biden Jr    133
Name: count, dtype: int64
party
Democrat       803
Republican     

In [197]:
# Checking for missing variables
for var in df.columns:
    missing_percentage = df[var].isna().sum() / len(df) * 100
    print(f"{var}: {missing_percentage:.2f}% missing")

# None of the variables appear to have any missing values

year: 0.00% missing
countyName: 0.00% missing
countyCode: 0.00% missing
candidate: 0.00% missing
party: 0.00% missing
candidatevotes: 0.00% missing
totalvotes: 0.00% missing


In [198]:
# No missing values!
df.isna().sum()

year              0
countyName        0
countyCode        0
candidate         0
party             0
candidatevotes    0
totalvotes        0
dtype: int64

In [199]:
df.dtypes

year               int64
countyName        object
countyCode         int64
candidate         object
party             object
candidatevotes     int64
totalvotes         int64
dtype: object

In [200]:
# Mount cleaned data to Drive
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
df.to_csv('/content/drive/My Drive/Spring 2024/DS 3001/DS3001 Projects/Voting Project/cleaned_voting_VA.csv')

Mounted at /content/drive
